In [45]:
# Install Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
# Download and extract Spark
!wget -q https://downloads.apache.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz
!tar xf spark-3.5.4-bin-hadoop3.tgz
# Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.4-bin-hadoop3"
# Install findspark
!pip install -q findspark
# Initialize findspark
import findspark
findspark.init("/content/spark-3.5.4-bin-hadoop3")
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrame Basics").getOrCreate()
# Verify Spark installation
spark

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [58]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [61]:
# Download the CSV file locally using wget
!wget -O home_sales_revised.csv "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Read the locally downloaded CSV file into a DataFrame
df = spark.read.option("header", "true").csv("home_sales_revised.csv", inferSchema=True)

# Verify the DataFrame
df.show()

--2025-02-25 03:22:41--  https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv
Resolving 2u-data-curriculum-team.s3.amazonaws.com (2u-data-curriculum-team.s3.amazonaws.com)... 52.217.90.228, 3.5.7.115, 16.182.96.1, ...
Connecting to 2u-data-curriculum-team.s3.amazonaws.com (2u-data-curriculum-team.s3.amazonaws.com)|52.217.90.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2747874 (2.6M) [text/csv]
Saving to: ‘home_sales_revised.csv’

home_sales_revised. 100%[===================>]   2.62M  --.-KB/s    in 0.1s    

2025-02-25 03:22:41 (19.2 MB/s) - ‘home_sales_revised.csv’ saved [2747874/2747874]

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--

In [62]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

# Verifying the temporary view by running a simple query
spark.sql("SELECT * FROM home_sales LIMIT 10").show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [64]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
query = """
SELECT YEAR(date) as year, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
"""
result = spark.sql(query)

result.show()


+----+---------+
|year|avg_price|
+----+---------+
|2022|296363.88|
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
+----+---------+



In [65]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
"""
result = spark.sql(query)

result.show()

+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015| 288770.3|
|      2013|295962.27|
|      2014|290852.27|
|      2012|293683.19|
|      2016|290555.07|
|      2010|292859.62|
|      2011|291117.47|
|      2017|292676.79|
+----------+---------+



In [66]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
query = """
SELECT date_built, ROUND(AVG(price), 2) as avg_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
GROUP BY date_built
"""
result = spark.sql(query)

result.show()


+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2015|297609.97|
|      2013|303676.79|
|      2014|298264.72|
|      2012|307539.97|
|      2016| 293965.1|
|      2010|285010.22|
|      2011|276553.81|
|      2017|280317.58|
+----------+---------+



In [67]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
# Define the query
query = """
SELECT view, ROUND(AVG(price), 2) as avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
start_time = time.time()

result = spark.sql(query)

result.show()

# Calculate the runtime
runtime = time.time() - start_time
print(f"--- {runtime:.2f} seconds ---")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.06 seconds ---


In [68]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

Table 'home_sales' is cached: True


In [69]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [70]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) as avg_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""
result_cached = spark.sql(query)

result_cached.show()

runtime_cached = time.time() - start_time
print(f"--- {runtime_cached:.2f} seconds ---")



+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 2.01 seconds ---


In [71]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("home_sales_parquet")


In [72]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("home_sales_parquet")

In [73]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [74]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query = """
SELECT view, ROUND(AVG(price), 2) as avg_price
FROM home_sales_parquet
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view DESC
"""

result_parquet = spark.sql(query)

result_parquet.show()

runtime_parquet = time.time() - start_time
print(f"--- {runtime_parquet:.2f} seconds ---")


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.10 seconds ---


In [75]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [77]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")
print(f"Table 'home_sales' is cached: {is_cached}")

Table 'home_sales' is cached: False
